# BoZhon Robotic Kinematic Analysis
> Author: ZYL

In [1]:
# import
import roboticstoolbox as rtb
from roboticstoolbox.robot.DHLink import PrismaticDH, RevoluteDH
from spatialmath import *
import numpy as np
import math
import sympy as sp
import random
# 改变后端，弹窗显示
%matplotlib qt5

In [2]:
# 标准的运动学模型
BZRobot = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),
    PrismaticDH(a=0, alpha=0, theta=0, offset=0),
    RevoluteDH(a=0, alpha=0, d=0),
], name="BoZhonRobot")

In [3]:
# 带入误差的运动学模型
epislon = 0.01
BZRobot_real = rtb.DHRobot([
    PrismaticDH(a=epislon, alpha=math.pi/2+epislon, theta=math.pi/2+epislon, offset=epislon),
    PrismaticDH(a=epislon, alpha=math.pi/2+epislon, theta=math.pi/2+epislon, offset=epislon),
    PrismaticDH(a=epislon, alpha=0+epislon, theta=0, offset=epislon),
    RevoluteDH(a=0, alpha=0, d=0),
], name="BoZhonRobot")

In [4]:
a = sp.Symbol("a")
print(BZRobot_real.fkine([a, a, -a, 0]).A)
# BZRobot_real.plot([a, a, -a, 0]);

[[0.0100988300508766 0.999502056363797 -0.0298940287707309
  1.01979768343424*a + 0.00990200149999922]
 [-0.00989934167974705 0.0299940204376503 0.999501056413796
  0.0198990166738667 - 0.989751683636164*a]
 [0.999900003333289 -0.00979786009940893 0.0101973168804974
  0.979901336614957*a + 0.0299989900338329]
 [0 0 0 1.00000000000000]]


In [5]:
print(BZRobot.fkine([1, 1, 1, 0]).A)
BZRobot.plot([1, 1, 1, 0]);

[[-6.1232340e-17  1.0000000e+00  1.2246468e-16  1.0000000e+00]
 [ 6.1232340e-17 -1.2246468e-16  1.0000000e+00  1.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17 -6.1232340e-17  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [6]:
# 旋转误差器，目前至考虑旋转带来的误差，所有的偏置均为0
delta = 1e-4
coeff = 1
Rotator = rtb.DHRobot([
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
], name="Rotator")
Rotator.plot([delta, delta, delta]);
R_error = Rotator.fkine([delta, delta, delta]).A
R_correct = Rotator.fkine([0, 0, 0]).A
print(R_correct)
print(R_error - R_correct)
# 这个结果完美符合理论公式

[[ 1.0000000e+00  1.2246468e-16 -1.2246468e-16  1.0000000e+00]
 [-1.2246468e-16  1.0000000e+00  1.2246468e-16  1.0000000e+00]
 [ 1.2246468e-16 -1.2246468e-16  1.0000000e+00  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-1.00009999e-08 -9.99999993e-05  1.00009999e-04 -1.00004999e-04]
 [ 1.00009999e-04 -9.99999994e-09 -9.99899988e-05  9.99899998e-05]
 [-9.99999993e-05  9.99999998e-05 -9.99999994e-09  9.99999998e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [7]:
# 再用符号法进行验算
e_x, e_y, e_z = sp.Symbol("\epsilon_x"), sp.Symbol("\epsilon_y"), sp.Symbol("\epsilon_z")

In [8]:
delta = 1e-4
Rotator = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, offset=1, theta=math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=0.5, offset=math.pi),
    RevoluteDH(a=0, alpha=math.pi/2, d=0.5, offset=-math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=0.5, offset=math.pi),
], name="Rotator")
Rotator.plot([0, delta, delta, delta]);
R_error = Rotator.fkine([0, delta, delta, delta]).A
R_correct = Rotator.fkine([0, 0, 0, 0]).A
print(R_correct)
print(R_error - R_correct)

[[ 1.00000000e+00  1.98247506e-32 -1.22464680e-16  5.00000000e-01]
 [-4.82715273e-33  1.00000000e+00  1.22464680e-16  5.00000000e-01]
 [ 1.22464680e-16 -1.22464680e-16  1.00000000e+00  1.50000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.99999994e-09 -9.99999998e-05  9.99999993e-05 -4.99999999e-05]
 [ 1.00009999e-04 -9.99999994e-09 -9.99899993e-05 -5.00049999e-05]
 [-9.99899993e-05  9.99999993e-05 -9.99899996e-09  4.99974997e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [9]:
R_error = Rotator.fkine([0, e_x, e_y, e_z, 0]).A
# print(R_error)
res01 = sp.simplify(R_error[0][1])
# R_correct = Rotator.fkine([0, 0, 0, 0, 0]).A
# print(R_correct)
# print(R_error - R_correct)
res01 = res01.subs([(sp.sin(e_x), e_x), (sp.sin(e_y), e_y), (sp.sin(e_z), e_z)])
res01 = res01.subs([(sp.cos(e_x), 1), (sp.cos(e_y), 1), (sp.cos(e_z), 1)])
# 结果过于复杂，不要这么搞了

ValueError: cannot reshape array of size 5 into shape (4)

In [ ]:
res01

6.12323399573677e-17*\epsilon_z*((3.74939945665464e-33*\epsilon_x + 1.0)*sin(\epsilon_y - 1.5707963267949) - 6.12323399573677e-17*sqrt(2)*cos(\epsilon_x + pi/4)*cos(\epsilon_y - 1.5707963267949)) - 7.49879891330929e-33*\epsilon_z*(6.12323399573677e-17*(3.74939945665464e-33*\epsilon_x + 1.0)*cos(\epsilon_y - 1.5707963267949) + 6.12323399573677e-17*sin(\epsilon_x + 3.14159265358979) + 3.74939945665464e-33*sqrt(2)*sin(\epsilon_y - 1.5707963267949)*cos(\epsilon_x + pi/4) + 6.12323399573677e-17*cos(\epsilon_x + 3.14159265358979) + 6.12323399573677e-17) - 7.49879891330929e-33*(3.74939945665464e-33*\epsilon_x + 1.0)*sin(\epsilon_y - 1.5707963267949) - 1.0*(3.74939945665464e-33*\epsilon_x + 1.0)*cos(\epsilon_y - 1.5707963267949) - 6.12323399573677e-17*sqrt(2)*sin(\epsilon_y - 1.5707963267949)*cos(\epsilon_x + pi/4) + 4.59169004331693e-49*sqrt(2)*cos(\epsilon_x + pi/4)*cos(\epsilon_y - 1.5707963267949)

In [ ]:
# 平移误差器，目前仅考虑平移带来的误差
delta = 1e-4
Prismator = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
], name="Prismator")
Prismator.plot([delta, delta, delta]);
Prismator.fkine([delta, delta, delta])
# 完美符合理论公式

   1         0         0         0.0001    
   0         1         0         0.0001    
   0         0         1         0.0001    
   0         0         0         1         


总结：
- 旋转误差器
~~~python
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_z
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_x
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_y
~~~
- 平移误差器
~~~python
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dz
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dx
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dy
~~~

In [10]:
# 整体的运动学模型
coeff = 0
BZRobot = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0), # LINK1
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),   # LINK2
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    PrismaticDH(a=0, alpha=0, theta=0, offset=0),   # LINK3
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=0, d=0),  # LINK4
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
], name="BZRobot")

In [11]:
BZRobot.plot([1]+[0]*6+[1]+[0]*6+[1]+[0]*6+[0]+[0]*6);

In [12]:
from error import *

# 忽略所有高阶小量


In [13]:
# Set value
L1_error = [d_z1, d_x1, d_y1, e_z1, e_x1, e_y1]
L2_error = [d_z2, d_x2, d_y2, e_z2, e_x2, e_y2]
L3_error = [d_z3, d_x3, d_y3, e_z3, e_x3, e_y3]
L4_error = [d_z4, d_x4, d_y4, t_z, e_x4, e_y4]  # 注意，e_z4已经被弃用
paraList = [p_z] + L1_error + [p_x] + L2_error + [p_y] + L3_error + [e_z4] + L4_error
len(paraList)

28

In [14]:
L1_error = [0.1]*6
L2_error = [0.1]*6
L3_error = [0.1]*6
L4_error = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [15]:
# NOTE!
# 注意：由于坐标系不同，所以误差的角标不加区分

# 位置误差控制在0.001m以内
displace_error = 0.001 * random.uniform(0, 1)
# 角度误差控制在1度，即0.017弧度以内
angle_error = 0.017 * 1 * 0.5 * random.uniform(0, 1)
# 末端为吸盘端，认为主要的角度误差产生在z轴上
angle_error_z = 0.017/2 * 0.5 * random.uniform(0, 1)
angle_error_x = 0.017/2 * 0.5 * random.uniform(0, 1)
angle_error_y = 0.017/2 * 0.5 * random.uniform(0, 1)

valueDic = {
    a1: 0, b1: 0, c1: 0,
    a2: 0, b2: 0, c2: 0,
    a3: 0, b3: 0, c3: 0,

    p_x: 0.1, p_y: 0.1, p_z: 0.1,
    
    e_x1: angle_error, e_y1: angle_error, e_z1: angle_error,
    e_x2: angle_error, e_y2: angle_error, e_z2: angle_error,
    e_x3: angle_error, e_y3: angle_error, e_z3: angle_error,
    e_x4: angle_error_x, e_y4: angle_error_x, 
    e_z4: 0,    # NOTE! e_z4已经被弃用!!! 设置为0！！！
    d_x1: displace_error, d_y1: displace_error, d_z1: displace_error,
    d_x2: displace_error, d_y2: displace_error, d_z2: displace_error,
    d_x3: displace_error, d_y3: displace_error, d_z3: displace_error,
    d_x4: displace_error, d_y4: displace_error, d_z4: displace_error,
    t_z: angle_error_z
}

In [16]:
errorList = [
    (d_x1, valueDic[d_x1]), (d_y1, valueDic[d_y1]), (d_z1, valueDic[d_z1]),
    (d_x2, valueDic[d_x2]), (d_y2, valueDic[d_y2]), (d_z2, valueDic[d_z2]),
    (d_x3, valueDic[d_x3]), (d_y3, valueDic[d_y3]), (d_z3, valueDic[d_z3]),
    (d_x4, valueDic[d_x4]), (d_y4, valueDic[d_y4]), (d_z4, valueDic[d_z4]),
    (e_x1, valueDic[e_x1]), (e_y1, valueDic[e_y1]), (e_z1, valueDic[e_z1]),
    (e_x2, valueDic[e_x2]), (e_y2, valueDic[e_y2]), (e_z2, valueDic[e_z2]),
    (e_x3, valueDic[e_x3]), (e_y3, valueDic[e_y3]), (e_z3, valueDic[e_z3]),
    (e_x4, valueDic[e_x4]), (e_y4, valueDic[e_y4]), (e_z4, valueDic[e_z4]),
    (t_z, valueDic[t_z])
]
# 控制量
pList = [
    (p_x, valueDic[p_x]), (p_y, valueDic[p_y]), (p_z, valueDic[p_z]),

]
# 几何初始值（全部收缩在原点）
geo_list = [
    (a1, 0), (b1, 0), (c1, 0),
    (a2, 0), (b2, 0), (c2, 0),
    (a3, 0), (b3, 0), (c3, 0),
]

In [17]:
expect_displacement = [valueDic[p_x]]+[0]*6+[valueDic[p_y]]+[0]*6+[valueDic[p_z]]+[0]*6+[0]+[0]*6
print(BZRobot.fkine(expect_displacement))   # 目标位置
mat_sim_exp = BZRobot.fkine(expect_displacement).A
BZRobot.plot(expect_displacement);
x_sim_exp = mat_sim_exp[0][3]
y_sim_exp = mat_sim_exp[1][3]
z_sim_exp = mat_sim_exp[2][3]
print(x_sim_exp, y_sim_exp, z_sim_exp)


   0         1         0         0.1       
   0         0         1         0.1       
   1         0         0         0.1       
   0         0         0         1         

0.10000000000000006 0.09999999999999998 0.09999999999999998


In [18]:
real_displacement = []
for symbol in paraList:
    print(symbol, sep=' ', end=' ')
    real_displacement.append(valueDic[symbol])
    
len(real_displacement)

p_z \delta_z1 \delta_x1 \delta_y1 \epsilon_z1 \epsilon_x1 \epsilon_y1 p_x \delta_z2 \delta_x2 \delta_y2 \epsilon_z2 \epsilon_x2 \epsilon_y2 p_y \delta_z3 \delta_x3 \delta_y3 \epsilon_z3 \epsilon_x3 \epsilon_y3 \epsilon_z4 \delta_z4 \delta_x4 \delta_y4 theta_z \epsilon_x4 \epsilon_y4 

28

In [19]:
mat_sim_real = BZRobot.fkine(real_displacement).A
print(mat_sim_real)
BZRobot.plot(real_displacement);
x_sim_real = mat_sim_real[0][3]
y_sim_real = mat_sim_real[1][3]
z_sim_real = mat_sim_real[2][3]
print(x_sim_real, y_sim_real, z_sim_real)

[[ 0.00871288  0.9999254  -0.00856031  0.10249441]
 [-0.00857555  0.00863504  0.99992595  0.10307047]
 [ 0.99992527 -0.00863882  0.00865014  0.10307308]
 [ 0.          0.          0.          1.        ]]
0.10249441023252628 0.10307047274879771 0.10307308207130765


In [20]:
# 仿真得到的误差
print("仿真得到的误差")
x_sim_error = x_sim_real - x_sim_exp
y_sim_error = y_sim_real - y_sim_exp
z_sim_error = z_sim_real - z_sim_exp
print(x_sim_error, y_sim_error, z_sim_error)

仿真得到的误差
0.002494410232526223 0.003070472748797737 0.003073082071307673


In [21]:
# 理论计算的误差
P_err = P_err.subs(errorList + pList + geo_list)
print("理论计算的误差")
x_theory_error = P_err[0]
y_theory_error = P_err[1]
z_theory_error = P_err[2]
print(x_theory_error, y_theory_error, z_theory_error)

理论计算的误差
0.00287033093173126 0.00288363752669937 0.00288405615443609


In [22]:
# 计算两者偏差
print("计算两者偏差")
x_bias_percent = (x_sim_error - x_theory_error) / x_theory_error
y_bias_percent = (y_sim_error - y_theory_error) / y_theory_error
z_bias_percent = (z_sim_error - z_theory_error) / z_theory_error
print(x_bias_percent, y_bias_percent, z_bias_percent)

计算两者偏差
-0.130967720498452 0.0647915073820754 0.0655416908512120
